In [1]:
import torch
from torch.nn import DataParallel
from torch.utils.data import DataLoader
from torch.optim import AdamW

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import gc
from tqdm import tqdm
import sys
sys.path.append("../src")
from config import read_config, update_config
from metric import cal_mae_metric
from dataset import read_data
from util import smart_avg

In [2]:
config = read_config("Base")
config.gpu = [0]
config = update_config(config)

Using device: cuda
Number of device: 1
Model Output Folder: /home/vincent/Kaggle/GBVPP/output/Base/


In [3]:
subs = {
    "LSTM5_REG_PL": "../output/LSTM5_REG_PL/test_pred_all_18832.csv",
    "LSTM5_CLS_DO02_OP01_PL": "../output/LSTM5_CLS_DO02_OP01_PL/test_pred_all_18722.csv"
}

In [4]:
train, test = read_data(config)

In [5]:
pressure_unique = np.load("/home/vincent/Kaggle/GBVPP/input/pressure_unique.npy")

In [6]:
sub_all = None
for key, sub_file in tqdm(subs.items()):
    train[key] = 0
    tmp = pd.read_csv(sub_file)
    if sub_all is None:
        sub_all = tmp[["id","pressure"]].copy()
        sub_all["pressure"] = 0
        
    for fold in range(5):
        train.loc[train.query(f"fold=={fold}").index,key] = torch.load(f'../output/{key}/Fold_{fold}_best_model.pth')['valid_preds'].flatten()
        sub_all[key+f"_{fold}"] = tmp[f"preds_fold{fold}"]

    if sub_all is None:
        sub_all =  tmp.copy()
    
train["pressure_smart"] = smart_avg(train[list(subs.keys())], axis=1)
train["pressure_smart"] = train["pressure_smart"].map(lambda x: pressure_unique[np.abs(pressure_unique-x).argmin()])
CVs = train.groupby("fold").apply(lambda df: cal_mae_metric(df["pressure"],df["pressure_smart"],1-df["u_out"]))
print("Ensemble CV Median: Mean {:.4f}, std {:.4f}".format(np.mean(CVs), np.std(CVs)))

preds_cols = [key+f"_{i}"  for key in subs.keys() for i in range(5)]
sub_all["pressure"] = smart_avg(sub_all[preds_cols], axis=1)
sub_all["pressure"] = sub_all["pressure"].map(lambda x: pressure_unique[np.abs(pressure_unique-x).argmin()])
sub_all

100%|█████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.89s/it]


Inliers:  2193854 -> compute mean
Outliers: 3842146 -> compute median
Total:    6036000
Ensemble CV Median: Mean 0.1773, std 0.0016
Inliers:  1189049 -> compute mean
Outliers: 2834951 -> compute median
Total:    4024000


,id,pressure,LSTM5_REG_PL_0,LSTM5_REG_PL_1,LSTM5_REG_PL_2,LSTM5_REG_PL_3,LSTM5_REG_PL_4,LSTM5_CLS_DO02_OP01_PL_0,LSTM5_CLS_DO02_OP01_PL_1,LSTM5_CLS_DO02_OP01_PL_2,LSTM5_CLS_DO02_OP01_PL_3,LSTM5_CLS_DO02_OP01_PL_4
0,1,6.259305,6.189002,6.189002,6.189002,6.329606,6.259304,6.259305,6.259305,6.259305,6.259305,6.259305
1,2,5.978096,5.907794,5.978096,6.048398,5.978096,6.048398,5.978096,5.978096,5.907794,5.978096,5.978096
2,3,7.102930,7.102930,7.173232,7.173232,7.102930,7.102930,7.102930,7.102930,7.173232,7.173232,7.032628
3,4,7.665347,7.595045,7.665347,7.595045,7.665347,7.665347,7.665347,7.665347,7.665347,7.665347,7.595045
4,5,9.141693,9.141692,9.141692,9.141692,9.141692,9.071390,9.141693,9.141693,9.071390,9.141693,9.141693
...,...,...,...,...,...,...,...,...,...,...,...,...
4023995,4023996,5.275075,0.002414,0.002414,0.002414,0.002414,0.002414,21.233662,10.547735,14.695562,18.140367,15.257979
4023996,4023997,5.275075,0.002414,0.002414,0.002414,0.002414,0.002414,21.163359,10.547735,14.695562,18.070065,15.398583
4023997,4023998,5.345377,0.002414,0.002414,0.002414,0.002414,0.002414,21.163359,10.758642,14.695562,18.140367,15.398583
4023998,4023999,5.275075,0.002414,0.002414,0.002414,0.002414,0.002414,21.163359,10.547735,14.484656,18.702784,15.328281


In [7]:
sub_all.shape, train.shape

((4024000, 12), (6036000, 12))

In [8]:
sub_all[["id","pressure"]].to_csv("../output/ensemble_1030_smart_1773.csv",index=False)

In [14]:
sub_1366 = pd.read_csv("../output/submission_1366.csv")

In [16]:
sub_all["pressure"] = ((sub_all["pressure"] + sub_1366["pressure"])/2).map(lambda x: pressure_unique[np.abs(pressure_unique-x).argmin()])

In [17]:
sub_all[["id","pressure"]].to_csv("../output/ensemble_1028_smart_w1366.csv",index=False)

In [ ]:
# CONVERT TRAIN TO SERIES
series = train.groupby('breath_id').collect().reset_index()
for k in range(80): series[f'x_{k}'] = series.u_in.list.get(k)
for k in range(80): series[f'y_{k}'] = series.pressure.list.get(k)
for k in range(80): series[f'z_{k}'] = 1-series.u_out.list.get(k)
series.R = series.R.list.get(0)
series.C = series.C.list.get(0)
series = series.drop(['id','time_step','u_in','u_out','pressure'],axis=1)
series = series.merge(exhale,on='breath_id',how='left')
series = series.merge(length,on='breath_id',how='left')
series = series.rename({'time_step':'time_length','u_out':'expire'},axis=1)
series = series.sort_values('breath_id').reset_index(drop=True)

print('Train as series shape:', series.shape )
print('Min inhale length=', series['expire'].min(),',Max inhale length=', series['expire'].max(),
      'Max breath length=',series['time_length'].max() )
series.head()

In [ ]:
from cuml.neighbors import NearestNeighbors 

# FIRST 3 COLUMNS OF TRAIN SERIES DATAFRAME NEED TO BE IGNORED
IGNORE = 3
# MORE NEIGHBORS THAN WE NEED FOR ALL DISPLAYS
NEIGHBORS = 500
# HOW MANY TIME_STEPS TO USE FOR KNN COMPARISON
TIME_STEPS = 80
# HOW MANY INITIAL TIME_STEPS TO SKIP FOR KNN COMPARISON
SKIP = 0 

model = NearestNeighbors(n_neighbors=NEIGHBORS, metric='l1')
model.fit(series.iloc[:,IGNORE+SKIP:IGNORE+TIME_STEPS+SKIP])

SHOW = [24,68,75]
CTS = [300]*len(SHOW)

for USE,CT in zip(SHOW,CTS):
    distances, indices = model.kneighbors(series.iloc[USE:USE+1,IGNORE+SKIP:IGNORE+TIME_STEPS+SKIP])
        
    plt.figure(figsize=(20,4))
    plt.plot(np.arange(80),series.iloc[USE,IGNORE:IGNORE+80].to_array(),label='u_in')
    plt.plot(np.arange(80),series.iloc[USE,IGNORE+80:IGNORE+160].to_array(),label='pressure')
    y_max = plt.ylim()[1]
    if TIME_STEPS!=80: plt.plot([SKIP+TIME_STEPS-1,SKIP+TIME_STEPS-1],[0,y_max],'--',color='gray')
    if SKIP!=0: plt.plot([SKIP-1,SKIP-1],[0,y_max],'--',color='gray')
    exhale = series.loc[USE,'expire']
    plt.plot([exhale,exhale],[0,y_max],'--',color='black',label='exhale')
    rr = series.loc[USE,'R']; cc = series.loc[USE,'C']; 
    bb = series.loc[USE,'breath_id']; tt = series.loc[USE,'time_length']
    plt.title(f'Breath_Id={bb}, R={rr}, C={cc}, Length={tt:.3}',size=16)
    plt.legend()
    
    temp2 = series.iloc[indices.iloc[0].values].reset_index(drop=True)
    cdict = {5: 'yellow', 20: 'orange', 50: 'red'}
    plt.figure(figsize=(20,10))
    legend = {5:0, 20:0, 50:0}
    for r in range(CT):
        if r==0: plt.plot(np.arange(80), temp2.iloc[r,IGNORE:IGNORE+80].to_array(), color='blue', label='u_in')
        else: plt.plot(np.arange(80), temp2.iloc[r,IGNORE:IGNORE+80].to_array(), color='blue')
        if legend[temp2.loc[r,'R']]==0:
            legend[temp2.loc[r,'R']]=1; cc = temp2.loc[r,'R']
            plt.plot(np.arange(80), temp2.iloc[r,IGNORE+80:IGNORE+160].to_array(), 
                     c=cdict[temp2.loc[r,'R']], label=f'R={cc}')
        else: plt.plot(np.arange(80), temp2.iloc[r,IGNORE+80:IGNORE+160].to_array(), c=cdict[temp2.loc[r,'R']])
    y_max = plt.ylim()[1]
    if TIME_STEPS!=80: plt.plot([SKIP+TIME_STEPS-1,SKIP+TIME_STEPS-1],[0,y_max],'--',color='gray')
    if SKIP!=0: plt.plot([SKIP-1,SKIP-1],[0,y_max],'--',color='gray')
    plt.title(f'{CT} time series similar to breath_Id={bb}. For each new time series we plot and color code its pressure with respect to R variable',size=16)
    plt.legend()
    plt.show()
    
    temp2 = series.iloc[indices.iloc[0].values].reset_index(drop=True)
    cdict = {10: 'red', 20: 'orange', 50: 'yellow'}
    plt.figure(figsize=(20,10))
    legend = {10:0, 20:0, 50:0}
    for r in range(CT):
        if r==0: plt.plot(np.arange(80), temp2.iloc[r,IGNORE:IGNORE+80].to_array(), color='blue',label='u_in')
        else: plt.plot(np.arange(80), temp2.iloc[r,IGNORE:IGNORE+80].to_array(), color='blue')
        if legend[temp2.loc[r,'C']]==0:
            legend[temp2.loc[r,'C']]=1; cc = temp2.loc[r,'C']
            plt.plot(np.arange(80), temp2.iloc[r,IGNORE+80:IGNORE+160].to_array(), 
                     c=cdict[temp2.loc[r,'C']], label=f'C={cc}')
        else: plt.plot(np.arange(80), temp2.iloc[r,IGNORE+80:IGNORE+160].to_array(), c=cdict[temp2.loc[r,'C']])
    y_max = plt.ylim()[1]
    if TIME_STEPS!=80: plt.plot([SKIP+TIME_STEPS-1,SKIP+TIME_STEPS-1],[0,y_max],'--',color='gray')
    if SKIP!=0: plt.plot([SKIP-1,SKIP-1],[0,y_max],'--',color='gray')
    plt.title(f'{CT} time series similar to breath_Id={bb}. For each new time series we plot and color code its pressure with respect to C variable',size=16)
    plt.legend()
    plt.show()